In [3]:
import tkinter as tk
from tkinter import messagebox
import webbrowser
import time

# -----------------------------
# SPLASH SCREEN (from RTFED code)
# -----------------------------
class SplashScreen:
    def __init__(self, root, duration=7000):
        """
        A splash screen covering the entire screen,
        displaying a large label, fading in, waiting,
        and fading out over 'duration' ms total.
        """
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 0)  # transparent initially

        # Get screen dimensions
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()

        # Cover the entire screen
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        self.root.configure(bg="black")

        # Center the text (logo)
        self.label_text = tk.Label(
            self.root,
            text="McCutcheonLab Technologies\nMiceSplit",
            font=("Cascadia Code", 40, "bold"),
            bg="black",
            fg="violet"
        )
        self.label_text.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

        # Start fade in/out
        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        """
        Fade in for 1 second, then fade out for (duration - 1000) ms.
        """
        fade_in_time = 1000
        # after fade_in finishes, fade_out starts
        self.fade_in(fade_in_time, lambda: self.fade_out(duration - fade_in_time, self.close))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        steps = max(1, time_ms // 50)
        increment = 1.0 / steps

        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        steps = max(1, time_ms // 50)
        decrement = 1.0 / steps

        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def close(self):
        self.root.destroy()

# -----------------------------
# MICE SPLIT (Mice Distribution) APP
# -----------------------------
class MiceSplitApp:
    def __init__(self, root):
        self.root = root
        self.root.title("MiceSplit")

        # Data structure to store (mouse_id, weight)
        self.mice_data = []

        # Widgets
        # 1) Number of Groups
        self.lbl_groups = tk.Label(root, text="Number of groups needed:")
        self.lbl_groups.grid(row=0, column=0, padx=5, pady=5, sticky=tk.E)
        self.entry_groups = tk.Entry(root)
        self.entry_groups.grid(row=0, column=1, padx=5, pady=5)

        # 2) Fields for Mouse ID and Weight
        self.lbl_mouse_id = tk.Label(root, text="Mouse ID:")
        self.lbl_mouse_id.grid(row=1, column=0, padx=5, pady=5, sticky=tk.E)
        self.entry_mouse_id = tk.Entry(root)
        self.entry_mouse_id.grid(row=1, column=1, padx=5, pady=5)

        self.lbl_weight = tk.Label(root, text="Weight:")
        self.lbl_weight.grid(row=2, column=0, padx=5, pady=5, sticky=tk.E)
        self.entry_weight = tk.Entry(root)
        self.entry_weight.grid(row=2, column=1, padx=5, pady=5)

        # 3) Buttons
        self.btn_add_mouse = tk.Button(root, text="Add Mouse", command=self.add_mouse)
        self.btn_add_mouse.grid(row=3, column=0, padx=5, pady=5, sticky=tk.E)

        self.btn_distribute = tk.Button(root, text="Distribute", command=self.distribute)
        self.btn_distribute.grid(row=3, column=1, padx=5, pady=5, sticky=tk.W)

        # 4) Results box
        self.text_results = tk.Text(root, width=60, height=15)
        self.text_results.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

        # 5) Footer with copyright and hyperlink
        self.footer_frame = tk.Frame(root)
        self.footer_frame.grid(row=5, column=0, columnspan=2, pady=5)

        self.footer_label = tk.Label(
            self.footer_frame,
            text="© 2025 McCutcheonLab | UiT | Norway |",
            fg="blue"
        )
        self.footer_label.pack(side=tk.LEFT, padx=(0, 5))

        self.link_label = tk.Label(
            self.footer_frame,
            text="Developed by Hamid Taghipourbibalan",
            fg="blue",
            cursor="hand2"
        )
        self.link_label.pack(side=tk.LEFT)
        self.link_label.bind("<Button-1>", self.open_hyperlink)

    def open_hyperlink(self, event=None):
        webbrowser.open_new("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/")

    def add_mouse(self):
        """
        Adds (mouse_id, weight) to self.mice_data.
        """
        mouse_id = self.entry_mouse_id.get().strip()
        weight_str = self.entry_weight.get().strip()

        if not mouse_id or not weight_str:
            messagebox.showerror("Input Error", "Please enter both Mouse ID and Weight.")
            return

        # Validate numeric weight
        try:
            weight = float(weight_str)
        except ValueError:
            messagebox.showerror("Input Error", "Weight must be a number.")
            return

        # Add to list
        self.mice_data.append((mouse_id, weight))

        # Clear fields
        self.entry_mouse_id.delete(0, tk.END)
        self.entry_weight.delete(0, tk.END)

        # Update results box
        self.text_results.insert(tk.END, f"Added: Mouse ID={mouse_id}, Weight={weight}\n")

    def distribute(self):
        """
        Distribute the mice into groups using a greedy approach:
         1) Sort descending by weight
         2) Always add next heaviest to the current lightest group
        """
        n_groups_str = self.entry_groups.get().strip()
        if not n_groups_str:
            messagebox.showerror("Input Error", "Please enter number of groups.")
            return

        try:
            n_groups = int(n_groups_str)
            if n_groups <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("Input Error", "Number of groups must be a positive integer.")
            return

        if len(self.mice_data) == 0:
            messagebox.showerror("No Data", "No mice data to distribute.")
            return

        groups = self.distribute_mice_by_weight(self.mice_data, n_groups)

        # Clear old text
        self.text_results.delete("1.0", tk.END)

        # Summaries
        group_weights = [sum(m[1] for m in group) for group in groups]
        group_sizes = [len(group) for group in groups]
        group_averages = [(group_weights[i]/group_sizes[i] if group_sizes[i] > 0 else 0)
                          for i in range(n_groups)]

        # Print the distribution
        for i, group in enumerate(groups, start=1):
            self.text_results.insert(tk.END, f"--- Group {i} ---\n")
            for mouse_id, w in group:
                self.text_results.insert(tk.END, f"  Mouse ID: {mouse_id}, Weight: {w}\n")
            self.text_results.insert(tk.END, f"  Total Weight: {group_weights[i-1]:.2f}\n")
            self.text_results.insert(tk.END, f"  Average Weight: {group_averages[i-1]:.2f}\n\n")

        # Differences
        min_total, max_total = min(group_weights), max(group_weights)
        diff_total = max_total - min_total
        self.text_results.insert(tk.END,
            f"Difference (Total) between lightest and heaviest group: {diff_total:.2f}\n")

        min_avg, max_avg = min(group_averages), max(group_averages)
        diff_avg = max_avg - min_avg
        self.text_results.insert(tk.END,
            f"Difference (Average) between lightest and heaviest group: {diff_avg:.2f}\n")

    def distribute_mice_by_weight(self, mice, n_groups):
        # Sort descending by weight
        mice_sorted = sorted(mice, key=lambda x: x[1], reverse=True)
        groups = [[] for _ in range(n_groups)]
        sums = [0.0]*n_groups

        # Assign each mouse to the group with the smallest total so far
        for mouse_id, wt in mice_sorted:
            idx = sums.index(min(sums))
            groups[idx].append((mouse_id, wt))
            sums[idx] += wt

        return groups

# -----------------------------
# MAIN
# -----------------------------
def main():
    # First, show the splash screen
    splash_root = tk.Tk()
    SplashScreen(splash_root, duration=7000)
    # After 7s, destroy splash
    splash_root.after(7000, splash_root.destroy)
    splash_root.mainloop()

    # Now launch the MiceSplit app
    root = tk.Tk()
    app = MiceSplitApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
